In [1]:
import pandas as pd
import numpy as np

from datetime import timedelta

URL = {'customer':'https://assets.datacamp.com/production/repositories/1646/datasets/c3a701a4729471ae0b92d8c300b470fd2ec0a73a/user_demographics_v1.csv',
       'purchase':'https://assets.datacamp.com/production/repositories/1646/datasets/5decd183ef3710475958bbc903160fd6354379d5/purchase_data_v1.csv',
       'revenue':'https://assets.datacamp.com/production/repositories/1646/datasets/3afb49cad9fb91c02b71b52a2ddc0071ea13764c/daily_revenue.csv',
       'paywall demographics':'https://assets.datacamp.com/production/repositories/1646/datasets/01054025eb094ac1086edf8d206b313b84d911c5/user_demographics_paywall.csv',
       'results':'https://assets.datacamp.com/production/repositories/1646/datasets/2751adce60684a03d8b4132adeadab8a0b95ee56/AB_testing_exercise.csv'
      }
customer=pd.read_csv(URL['customer'])
purchase=pd.read_csv(URL['purchase'])
revenue=pd.read_csv(URL['revenue'])
paywall=pd.read_csv(URL['paywall demographics'])
ab_test_results=pd.read_csv(URL['results'])

### clean customer data
# convert uid to integer type
customer.uid = customer.uid.astype('int')
# truncate reg_date column to date
customer.reg_date = pd.to_datetime(customer.reg_date).dt.date

### clean purchase data
# convert date column to datetime
purchase.date = pd.to_datetime(purchase.date)

### clean revenue data
#convert date to datetime and truncate to date
revenue.date = pd.to_datetime(revenue.date).dt.date

display(customer.head())
display(purchase.head())
display(revenue.head())

,uid,reg_date,device,gender,country,age
0,54030035,2017-06-29,and,M,USA,19
1,72574201,2018-03-05,iOS,F,TUR,22
2,64187558,2016-02-07,iOS,M,USA,16
3,92513925,2017-05-25,and,M,BRA,41
4,99231338,2017-03-26,iOS,M,FRA,59


,date,uid,sku,price
0,2017-07-10,41195147,sku_three_499,499
1,2017-07-15,41195147,sku_three_499,499
2,2017-11-12,41195147,sku_four_599,599
3,2017-09-26,91591874,sku_two_299,299
4,2017-12-01,91591874,sku_four_599,599


,index,gender,device,revenue,date,country
0,1,male,ios,186557.0,2013-09-20,USA
1,2,male,ios,177217.0,2013-09-21,USA
2,3,male,ios,177197.0,2013-09-22,USA
3,4,male,ios,157193.0,2013-09-23,USA
4,5,male,ios,165592.0,2013-09-24,USA


In [2]:
# Combine purchase and customer data
df=purchase.merge(customer, on=['uid'], how='inner')
# Summary statistics for price and age
purchase_summary = df.agg({'price':['mean','median'],
                            'age':['mean','median']})
print(purchase_summary)
# Summary statistics for price by device and gender
grouped_data = df.groupby(by=['device','gender'])
purchase_summary = grouped_data.agg({'price':['mean','median','std']})
print(purchase_summary)

             price        age
mean    406.772596  23.922274
median  299.000000  21.000000
                    price                   
                     mean median         std
device gender                               
and    F       400.747504    299  179.984378
       M       416.237308    499  195.001520
iOS    F       404.435330    299  181.524952
       M       405.272401    299  196.843197


## Calculating KPI

A KPI of interest would be average amount paid per purchase in a user's first month. This KPI can provide a sense of the popularity of different in-app purchase price points to users within their first month.

In [5]:
last_date - timedelta(days=28)

datetime.date(2018, 2, 17)

In [3]:
# Most current date
last_date = max(df.reg_date)
# Compute max_purchase_date
max_purchase_date = last_date - timedelta(days=28)
#print("max purchase date:", max_purchase_date)
# Filter out users who registered in the last 28 days.
df_filt = df[df.reg_date < max_purchase_date]
# Filter to contain only purchases within the first 28 days of registration
df_filt = df_filt[df_filt.date <= (df_filt.reg_date + timedelta(days=28))]
# Mean price paid per purchase
print("Mean price paid per purchase:", df_filt.price.mean())

Mean price paid per purchase: 414.4237288135593


Users paid an average of **414** cents in their first 28 days of registeration.

### Average purchase price by cohort

In [51]:
max_reg_date = max(df.reg_date) - timedelta(days=28)
# Month 1 values
month1 = np.where((df.reg_date < max_reg_date) & \
                  (df.date < df.reg_date + timedelta(days=28)),
                 df.price, np.NaN)
df['month1'] = month1
df_grouped = df.groupby(by=['gender','device'], as_index=False)
summary = df_grouped.agg({'month1':['mean','median'],
                'price':['mean','median']})
print(summary)

  gender device      month1              price       
                       mean median        mean median
0      F    and  388.204545  299.0  400.747504    299
1      F    iOS  432.587786  499.0  404.435330    299
2      M    and  413.705882  399.0  416.237308    499
3      M    iOS  433.313725  499.0  405.272401    299


Female android users tend to spend less in their first month, while female iOS users tend to spend more than in their first month.
Male android users tend to spend sligtly less than average in their first month, and male iOS users tend to spend significantly more in their first month.